In [2]:
import re
from time import sleep
import itertools
import requests
from pathlib import Path
from bs4 import BeautifulSoup
import pandas as pd
import pyarrow as pa
from zipfile import ZipFile
from multiprocessing.pool import ThreadPool

import dotenv
dotenv.load_dotenv(".envrc", override=True)

True

In [7]:
path = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/"
path = Path("/final/main")
print(path.name == "main")

True


In [18]:

# URLS = {
#     "hist_data": "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/historical/",
#     "recent_data": "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/recent/",
# }

def get_file_links(url: str, category: str) -> list:
        """Get urls of files"""
        tmp_url = url + category 
        page = requests.get(tmp_url)
        soup = BeautifulSoup(page.content, "html.parser")

        

        links = soup.find_all("a", href=re.compile(".pdf$|.txt$|.zip$"))
        print("found {} files for download".format(len(links)))
        
        return [[link, category] for link in links]


def download_files(link: list) -> None:
    link, category = link 
    print(link, category)

    save_path = Path(f"./data/{category}/download")
    save_path.mkdir(parents=True, exist_ok=True)
    
    file_path = save_path / link["href"]
    mode = "w+b" if "pdf" or "zip" in link["href"] else "w+"
    file_url = url + link["href"]
    if not file_path.is_file():
        with requests.get(file_url) as response:
            with open(str(file_path), mode) as file:
                file.write(response.content)
                sleep(0.5)
    
    print("download finished")
    # TODO: add error handler for timeout with too many requests
url = "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/daily/kl/"
category = "recent"
links = get_file_links(url, category)
# with ThreadPool(5) as pool:
#     for count, result in enumerate(pool.imap_unordered(download_files, links)):
#         print(f"downloads finished: {count}")


found 575 files for download
575


In [ ]:
for path in Path("data/").glob("*_data"):
        source_path = path / "download"
        source_path.mkdir(parents=True, exist_ok=True)
        target_data_path = path / "tables"
        target_data_path.mkdir(parents=True, exist_ok=True)
        target_metadata_path = path / "metadata"
        target_metadata_path.mkdir(parents=True, exist_ok=True)

        for file in source_path.glob("*.zip"):
            with ZipFile(file, "r") as zip:
                fname_data_lst = [
                    fname for fname in zip.namelist() if "produkt_klima_tag" in fname
                ]
                fname_metadata_lst = [
                    fname for fname in zip.namelist() if "Metadaten" in fname
                ]
                for fname in fname_data_lst:
                    zip.extract(fname, target_path)
                for fname in fname_metadata_lst:
                    zip.extract(fname, target_path)

        files_to_rmv = target_metadata_path.glob("*.html")
        for file in files_to_rmv:
            file.unlink()

        print("finished unzipping to folder {}".format(path))

## Load main dataset

In [ ]:
dtypes= {
    'STATIONS_ID': "string", 
    'QN_3': "UInt8", 
    'QN_4': "UInt8", 
    ' RSK': "Float32", 
    'RSKF': "UInt8", 
    'SHK_TAG': "UInt8",
    '  NM': "string", 
    ' TMK': "Float32", 
    ' UPM': "Float32", 
    ' TXK': "Float32", 
    ' TNK': "Float32", 
    ' TGK': "Float32",
}


df_new_lst = [pd.read_table(
        file, 
        sep=";",
        usecols=['STATIONS_ID', 'MESS_DATUM', 'QN_3', 'QN_4', ' RSK',
        'RSKF', 'SHK_TAG', '  NM', ' TMK', ' UPM',
        ' TXK', ' TNK', ' TGK'],
        dtype_backend="pyarrow",
        dtype=dtypes,
        na_values=None,
    ) for file in Path("./data/recent_data/extracts").glob("*.txt")]

df_hist_lst = [pd.read_table(
        file, 
        sep=";",
        usecols=['STATIONS_ID', 'MESS_DATUM', 'QN_3', 'QN_4', ' RSK',
        'RSKF', 'SHK_TAG', '  NM', ' TMK', ' UPM',
        ' TXK', ' TNK', ' TGK'],
        dtype_backend="pyarrow",
        dtype=dtypes,
        na_values=None,
    ) for file in Path("./data/hist_data/extracts").glob("*.txt")]

## Prepare main dataset

In [ ]:
# df = pd.concat(df_new_lst + df_hist_lst)

# df["MESS_DATUM"] = pd.to_datetime(df["MESS_DATUM"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT", nonexistent="NaT")
# df.columns = df.columns.str.replace(' ', '')
# df.info()

In [21]:
# df.to_parquet("./data/parquet/data.parquet")
# df = pd.read_parquet("./data/parquet/main.parquet", dtype_backend="pyarrow")
df.info()

df["year_month"] = df["mess_datum"].dt.strftime("%Y-%m")
df.to_parquet("./data/parquet/main/", partition_cols=["year_month"], compression="gzip")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17325569 entries, 0 to 17325568
Data columns (total 13 columns):
 #   Column       Dtype                                     
---  ------       -----                                     
 0   stations_id  string[pyarrow]                           
 1   mess_datum   timestamp[us, tz=Europe/Brussels][pyarrow]
 2   qn_3         uint8[pyarrow]                            
 3   qn_4         uint8[pyarrow]                            
 4   rsk          float[pyarrow]                            
 5   rskf         uint8[pyarrow]                            
 6   shk_tag      uint8[pyarrow]                            
 7   nm           string[pyarrow]                           
 8   tmk          float[pyarrow]                            
 9   upm          float[pyarrow]                            
 10  txk          float[pyarrow]                            
 11  tnk          float[pyarrow]                            
 12  tgk          float[pyarrow

## Load and prepare metadata

In [ ]:
# dtypes = {
#     'Stations_id': "string",
#     'Stationshoehe': "Float32",
#     'Geogr.Breite': "Float32",
#     'Geogr.Laenge': "Float32",
#     'von_datum': "string",
#     'bis_datum': "string",
#     'Stationsname': "string",
# }

# df_new_lst = [pd.read_table(
#         file, 
#         sep=";",
#         encoding="latin1",
#         dtype=dtypes,
#         dtype_backend="pyarrow",
#         na_values=None) for file in Path("./data/recent_data/metadata").glob("Metadaten_Geographie*")]

# df_hist_lst = [pd.read_table(
#         file, 
#         sep=";",
#         encoding="latin1",
#         dtype=dtypes,
#         dtype_backend="pyarrow",
#         na_values=None) for file in Path("./data/hist_data/metadata").glob("Metadaten_Geographie*")]

# df = pd.concat(df_new_lst + df_hist_lst)
# df.Stations_id = df.Stations_id.str.replace(' ', '')
# df = df.replace(-999, None).astype(dtypes)
# df["von_datum"] = pd.to_datetime(df["von_datum"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT")
# df["bis_datum"] = pd.to_datetime(df["bis_datum"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT")
# df.columns = df.columns.str.replace(' ', '')

In [ ]:
# df.to_parquet("./data/parquet/metadata_geo.parquet")
# df = pd.read_parquet("./data/parquet/metadata_geo.parquet", dtype_backend="pyarrow")
# df.info()

In [19]:
df_new_lst = []
for file in Path("./data/recent_data/metadata").glob("Metadaten_Stationsname_Betreibername*"):
    with open(file, "r", encoding="latin1") as txt_file:
                    text = txt_file.read()
                    table_with_footer = text.split("\n\nStations_ID;Betreibername;Von_Datum;Bis_Datum\n ")[1]
                    df_new_lst += [line.split(";") for line in table_with_footer.splitlines()[:-1]]

df = pd.DataFrame(df_new_lst, columns=["stations_id", "betreibername", "betrieb_von_datum", "betrieb_bis_datum"])    
df.stations_id = df.stations_id.str.replace(' ', '')
df["betrieb_von_datum"] = pd.to_datetime(df["betrieb_von_datum"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT")
df["betrieb_bis_datum"] = pd.to_datetime(df["betrieb_bis_datum"], format="%Y%m%d", errors="coerce", utc=False).dt.tz_localize("Europe/Brussels", ambiguous = "NaT")

In [1]:
# df.to_parquet("./data/parquet/metadata_operator.parquet")
# df = pd.read_parquet("./data/parquet/metadata_operator.parquet", dtype_backend="pyarrow")
# df.info()
test = ["a", "b", "c"]
if "a" in test:
    print("TRUE") 

TRUE


## Spark

In [ ]:
# from pyspark.sql import SparkSession
# import pyspark.sql.types as T
# import pyspark.sql.functions as F

# spark.conf.set('spark.sql.legacy.parquet.datetimeRebaseModeInRead', 'CORRECTED')

# schema = T.StructType([
#     T.StructField('STATIONS_ID', T.StringType(), False),
#     T.StructField('MESS_DATUM', T.TimestampType(), True),
#     T.StructField('QN_3', T.ShortType(), True),
#     T.StructField('QN_4', T.ShortType(), True),
#     T.StructField('RSK', T.DoubleType(), True),
#     T.StructField('RSKF', T.ShortType(), True),
#     T.StructField('SHK_TAG', T.IntegerType(), True),
#     T.StructField('NM', T.StringType(), True),
#     T.StructField('TMK', T.DoubleType(), True),
#     T.StructField('UPM', T.DoubleType(), True),
#     T.StructField('TXK', T.DoubleType(), True),
#     T.StructField('TNK', T.DoubleType(), True),
#     T.StructField('TGK', T.DoubleType(), True),
# ])

# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName('SparkNotebook') \
#     .getOrCreate()

In [ ]:
# data_path = Path("./data/data.parquet")

In [ ]:
# df = spark.read \
#     .option("header", "true") \
#     .parquet(str(data_path))

# df.createOrReplaceTempView("temps")


In [ ]:
# df.printSchema()

In [ ]:
# results = spark.sql("""
# SELECT distinct(extract(year from MESS_DATUM)) from temps
# where date(MESS_DATUM) >= "1900-01-01"
# order by 1 desc
# """)
# results.show(40, False)

In [ ]:
# results = spark.sql("""
# SELECT STATIONS_ID,extract(year from MESS_DATUM), max(TXK) as max_temp_2m from temps
# group by 1,2
# order by 3 desc
# """)
# results.show(40, False)